In [1]:
import json
from google.colab import files

uploaded = files.upload()

Saving 혐오탐지데이터 2만.jsonl to 혐오탐지데이터 2만.jsonl


In [2]:
file_name = list(uploaded.keys())[0]
print(f"업로드된 파일: {file_name}")

data = []
with open(file_name, 'r', encoding='utf-8-sig') as f:
    for line in f:
        data.append(json.loads(line))

print(f"데이터 개수: {len(data)}개")
print(f"첫 번째 데이터: {data[0]}")

업로드된 파일: 혐오탐지데이터 2만.jsonl
데이터 개수: 20022개
첫 번째 데이터: {'utterance': ['A: 오늘 지각할 뻔했다ㅋㅋ 버스 진짜 안 옴', 'B: 나도 거의 뛰어서 옴;;', 'A: 수학쌤 오늘 또 지옥 시간일 듯', 'B: 그 ㅆㅂ 늙은이 목소리만 들어도 현타 옴'], 'target_index': 3, 'label': 'toxic'}


In [3]:
# 2단계: 데이터 탐색하고 전처리하기

# 1. 데이터 구조 확인
print("=== 데이터 구조 확인 ===")
print(f"전체 데이터 개수: {len(data)}")
print(f"첫 번째 데이터 키들: {data[0].keys()}")
print()

# 2. 몇 개 샘플 확인
print("=== 샘플 3개 확인 ===")
for i in range(3):
    print(f"샘플 {i+1}:")
    print(f"  대화: {data[i]['utterance']}")
    print(f"  타겟 인덱스: {data[i]['target_index']}")
    print(f"  타겟 발언: {data[i]['utterance'][data[i]['target_index']]}")
    print(f"  라벨: {data[i]['label']}")
    print()

# 3. 라벨 분포 확인
print("=== 라벨 분포 ===")
toxic_count = 0
non_toxic_count = 0

for item in data:
    if item['label'] == 'toxic':
        toxic_count += 1
    else:
        non_toxic_count += 1

print(f"Toxic: {toxic_count}개")
print(f"Non-toxic: {non_toxic_count}개")
print(f"비율 - Toxic: {toxic_count/len(data)*100:.1f}%, Non-toxic: {non_toxic_count/len(data)*100:.1f}%")
print()

# 4. 기본 전처리 - 라벨을 숫자로 바꾸기
print("=== 전처리 시작 ===")
processed_data = []

for item in data:

    label = 1 if item['label'] == 'toxic' else 0

    full_conversation = " ".join(item['utterance'])

    target_sentence = item['utterance'][item['target_index']]

    processed_item = {
        'full_conversation': full_conversation,
        'target_sentence': target_sentence,
        'label': label
    }

    processed_data.append(processed_item)

print(f"전처리 완료! {len(processed_data)}개 데이터 준비됨")
print(f"첫 번째 전처리된 데이터:")
print(f"  전체 대화: {processed_data[0]['full_conversation'][:100]}...")
print(f"  타겟 발언: {processed_data[0]['target_sentence']}")
print(f"  라벨: {processed_data[0]['label']}")

=== 데이터 구조 확인 ===
전체 데이터 개수: 20022
첫 번째 데이터 키들: dict_keys(['utterance', 'target_index', 'label'])

=== 샘플 3개 확인 ===
샘플 1:
  대화: ['A: 오늘 지각할 뻔했다ㅋㅋ 버스 진짜 안 옴', 'B: 나도 거의 뛰어서 옴;;', 'A: 수학쌤 오늘 또 지옥 시간일 듯', 'B: 그 ㅆㅂ 늙은이 목소리만 들어도 현타 옴']
  타겟 인덱스: 3
  타겟 발언: B: 그 ㅆㅂ 늙은이 목소리만 들어도 현타 옴
  라벨: toxic

샘플 2:
  대화: ['A: 오늘 지각할 뻔했다ㅋㅋ 버스 진짜 안 옴', 'B: 나도 거의 뛰어서 옴;;', 'A: 수학쌤 오늘 또 지옥 시간일 듯', 'B: 수학쌤 목소리 너무 지루하긴 해']
  타겟 인덱스: 3
  타겟 발언: B: 수학쌤 목소리 너무 지루하긴 해
  라벨: non-toxic

샘플 3:
  대화: ['A: 급식 오늘 뭐냐?', 'B: 아마 제육볶음이었을걸?', 'A: 개좋다ㅋㅋ 밥 두 공기 각', 'B: 니 그 돼지같은 몸으로 또 퍼먹냐ㅋㅋ 역겹다 진짜']
  타겟 인덱스: 3
  타겟 발언: B: 니 그 돼지같은 몸으로 또 퍼먹냐ㅋㅋ 역겹다 진짜
  라벨: toxic

=== 라벨 분포 ===
Toxic: 10011개
Non-toxic: 10011개
비율 - Toxic: 50.0%, Non-toxic: 50.0%

=== 전처리 시작 ===
전처리 완료! 20022개 데이터 준비됨
첫 번째 전처리된 데이터:
  전체 대화: A: 오늘 지각할 뻔했다ㅋㅋ 버스 진짜 안 옴 B: 나도 거의 뛰어서 옴;; A: 수학쌤 오늘 또 지옥 시간일 듯 B: 그 ㅆㅂ 늙은이 목소리만 들어도 현타 옴...
  타겟 발언: B: 그 ㅆㅂ 늙은이 목소리만 들어도 현타 옴
  라벨: 1


In [4]:
from transformers import AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')




test_sentence = processed_data[0]['target_sentence']
tokens = tokenizer.tokenize(test_sentence)
token_ids = tokenizer.encode(test_sentence)
decoded = tokenizer.decode(token_ids)
inputs = tokenizer(
    test_sentence,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)
test_sentences = [item['target_sentence'] for item in processed_data[:3]]
for i, sent in enumerate(test_sentences):
    print(f"  {i+1}: {sent}")
batch_inputs = tokenizer(
    test_sentences,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]


=== 토큰화 테스트 ===
원본 문장: B: 그 ㅆㅂ 늙은이 목소리만 들어도 현타 옴
토큰화 결과: ['B', ':', '그', '[UNK]', '늙은이', '목소리', '##만', '들어도', '현', '##타', '옴']
토큰 ID: [2, 38, 30, 636, 1, 22184, 4243, 2154, 16265, 1919, 2256, 1449, 3]
디코딩 결과: [CLS] B : 그 [UNK] 늙은이 목소리만 들어도 현타 옴 [SEP]

=== 모델 입력용 토큰화 ===
input_ids 크기: torch.Size([1, 128])
attention_mask 크기: torch.Size([1, 128])
input_ids: tensor([[    2,    38,    30,   636,     1, 22184,  4243,  2154, 16265,  1919,
          2256,  1449,     3,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
          

In [5]:
# 4단계: BERT 모델 로드하고 분류기 만들기

from transformers import AutoModel
import torch.nn as nn
import torch

bert_model = AutoModel.from_pretrained('klue/bert-base')

class HateDetectionModel(nn.Module):
    def __init__(self, bert_model):
        super(HateDetectionModel, self).__init__()
        self.bert = bert_model
        self.classifier = nn.Linear(768, 2)
        self.dropout = nn.Dropout(0.1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

# 3. 모델 생성
print("분류 모델 생성 중...")
model = HateDetectionModel(bert_model)
print("분류 모델 생성 완료!")

# 4. 모델 구조 확인
print(f"\n=== 모델 구조 ===")
print(f"전체 파라미터 수: {sum(p.numel() for p in model.parameters()):,}")
print(f"학습 가능한 파라미터 수: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# 5. 모델 테스트 (forward pass)
print(f"\n=== 모델 테스트 ===")
model.eval()  # 평가 모드로 설정

# 앞서 만든 배치 데이터로 테스트
with torch.no_grad():
    logits = model(batch_inputs['input_ids'], batch_inputs['attention_mask'])

print(f"입력 크기: {batch_inputs['input_ids'].shape}")
print(f"출력 크기: {logits.shape}")
print(f"출력 예시: {logits}")

# 확률로 변환 (소프트맥스)
probabilities = torch.softmax(logits, dim=1)
print(f"확률 출력: {probabilities}")

# 예측 클래스
predictions = torch.argmax(logits, dim=1)
print(f"예측 클래스: {predictions}")
print(f"예측 의미: {['non-toxic' if p == 0 else 'toxic' for p in predictions]}")

print(f"\n✅ 4단계 완료! BERT 분류 모델 준비됨")

한국어 BERT 모델 로드 중...


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

BERT 모델 로드 완료!
분류 모델 생성 중...
분류 모델 생성 완료!

=== 모델 구조 ===
전체 파라미터 수: 110,618,882
학습 가능한 파라미터 수: 110,618,882

=== 모델 테스트 ===
입력 크기: torch.Size([3, 128])
출력 크기: torch.Size([3, 2])
출력 예시: tensor([[-0.6814, -0.7199],
        [-0.5466, -0.9273],
        [-0.4654, -0.8723]])
확률 출력: tensor([[0.5096, 0.4904],
        [0.5940, 0.4060],
        [0.6003, 0.3997]])
예측 클래스: tensor([0, 0, 0])
예측 의미: ['non-toxic', 'non-toxic', 'non-toxic']

✅ 4단계 완료! BERT 분류 모델 준비됨


In [6]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# 1. PyTorch Dataset 클래스 정의
class HateDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# 2. 데이터 준비
print("=== 데이터 준비 ===")
texts = [item['target_sentence'] for item in processed_data]
labels = [item['label'] for item in processed_data]

print(f"전체 텍스트 개수: {len(texts)}")
print(f"전체 라벨 개수: {len(labels)}")

# 3. 학습/검증 데이터 분할 (80:20)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels,
    test_size=0.2,
    random_state=42,
    stratify=labels  # 라벨 비율 유지
)

print(f"학습 데이터: {len(train_texts)}개")
print(f"검증 데이터: {len(val_texts)}개")
print(f"학습 데이터 toxic 비율: {sum(train_labels)/len(train_labels)*100:.1f}%")
print(f"검증 데이터 toxic 비율: {sum(val_labels)/len(val_labels)*100:.1f}%")

# 4. Dataset 객체 생성
print(f"\n=== Dataset 객체 생성 ===")
train_dataset = HateDataset(train_texts, train_labels, tokenizer)
val_dataset = HateDataset(val_texts, val_labels, tokenizer)

print(f"학습 데이터셋 크기: {len(train_dataset)}")
print(f"검증 데이터셋 크기: {len(val_dataset)}")

# 5. 데이터 샘플 확인
print(f"\n=== 데이터 샘플 확인 ===")
sample = train_dataset[0]
print(f"input_ids 크기: {sample['input_ids'].shape}")
print(f"attention_mask 크기: {sample['attention_mask'].shape}")
print(f"label: {sample['label']}")
print(f"샘플 텍스트: {train_texts[0]}")

# 6. DataLoader 생성
print(f"\n=== DataLoader 생성 ===")
BATCH_SIZE = 16

train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0  # 코랩에서는 0으로 설정
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

print(f"학습 DataLoader 배치 개수: {len(train_dataloader)}")
print(f"검증 DataLoader 배치 개수: {len(val_dataloader)}")

# 7. DataLoader 테스트
print(f"\n=== DataLoader 테스트 ===")
for batch in train_dataloader:
    print(f"배치 input_ids 크기: {batch['input_ids'].shape}")
    print(f"배치 attention_mask 크기: {batch['attention_mask'].shape}")
    print(f"배치 labels 크기: {batch['label'].shape}")
    print(f"배치 labels: {batch['label']}")
    break  # 첫 번째 배치만 확인

print(f"\n✅ 5단계 완료! 데이터 로딩 준비됨")

=== 데이터 준비 ===
전체 텍스트 개수: 20022
전체 라벨 개수: 20022
학습 데이터: 16017개
검증 데이터: 4005개
학습 데이터 toxic 비율: 50.0%
검증 데이터 toxic 비율: 50.0%

=== Dataset 객체 생성 ===
학습 데이터셋 크기: 16017
검증 데이터셋 크기: 4005

=== 데이터 샘플 확인 ===
input_ids 크기: torch.Size([128])
attention_mask 크기: torch.Size([128])
label: 0
샘플 텍스트: B: 좋은 계기가 될 수도 있으니까 시도해보는 거 괜찮아 보여

=== DataLoader 생성 ===
학습 DataLoader 배치 개수: 1002
검증 DataLoader 배치 개수: 251

=== DataLoader 테스트 ===
배치 input_ids 크기: torch.Size([16, 128])
배치 attention_mask 크기: torch.Size([16, 128])
배치 labels 크기: torch.Size([16])
배치 labels: tensor([0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0])

✅ 5단계 완료! 데이터 로딩 준비됨


In [10]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("사용 디바이스:", device)

import os, json, numpy as np, optuna, torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
from transformers import (
    AutoConfig, AutoTokenizer,
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup
)

# === 유틸 ===
def make_loaders(train_ds, val_ds, batch_size=16):
    return (
        DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=0),
        DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=0),
    )

@torch.no_grad()
def _eval_step(outputs, labels):
    if hasattr(outputs, "logits"):  # HF 모델
        logits = outputs.logits
    else:                           # 혹시 커스텀 모델 대비
        logits = outputs
    preds = torch.argmax(logits, dim=1)
    return preds

def run_epoch_cls(model, dataloader, optimizer, criterion, device, train_mode=True, scheduler=None, max_grad_norm=1.0):
    model.train() if train_mode else model.eval()
    total_loss = 0.0
    all_preds, all_labels = [], []

    for batch in dataloader:
        input_ids      = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels         = batch["label"].to(device)

        if train_mode:
            optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        if hasattr(outputs, "logits"):
            logits = outputs.logits
        else:
            logits = outputs
        loss = criterion(logits, labels)

        if train_mode:
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            if scheduler is not None:
                scheduler.step()

        total_loss += loss.item() * labels.size(0)
        preds = torch.argmax(logits, dim=1)
        all_preds.append(preds.detach().cpu().numpy())
        all_labels.append(labels.detach().cpu().numpy())

    avg_loss = total_loss / len(dataloader.dataset)
    all_preds  = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    p_bin, r_bin, f1_bin, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary',  zero_division=0)
    p_mac, r_mac, f1_mac, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro',   zero_division=0)
    acc = (all_preds == all_labels).mean()
    cm  = confusion_matrix(all_labels, all_preds, labels=[0,1]).tolist()

    return {
        "loss": avg_loss, "acc": acc,
        "p_bin": p_bin, "r_bin": r_bin, "f1_bin": f1_bin,
        "p_macro": p_mac, "r_macro": r_mac, "f1_macro": f1_mac,
        "cm": cm
    }

# === Optuna 목적함수: lr / epoch / batch_size 간단 탐색 ===
def objective(trial):
    lr         = trial.suggest_float("lr", 1e-5, 5e-5, log=True)
    epochs     = trial.suggest_int("epochs", 3, 5)
    batch_size = trial.suggest_categorical("batch_size", [16, 32])

    train_dl, val_dl = make_loaders(train_dataset, val_dataset, batch_size)
    id2label = {0: "non_toxic", 1: "toxic"}
    label2id = {"non_toxic": 0, "toxic": 1}
    config = AutoConfig.from_pretrained(
        "klue/bert-base", num_labels=2, id2label=id2label, label2id=label2id
    )
    model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", config=config).to(device)

    optimizer = optim.AdamW(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # 간단 스케줄러(워밍업 10%)
    total_steps = len(train_dl) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=int(0.1*total_steps), num_training_steps=total_steps
    )

    best_f1 = 0.0
    for _ in range(epochs):
        _  = run_epoch_cls(model, train_dl, optimizer, criterion, device, train_mode=True,  scheduler=scheduler)
        vm = run_epoch_cls(model, val_dl,   optimizer, criterion, device, train_mode=False, scheduler=None)
        best_f1 = max(best_f1, vm["f1_macro"])  # 기본: macro-F1 기준

    return best_f1

# === HPO 실행 (원하는 trial 수로) ===
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20, show_progress_bar=True)

print("\n=== HPO 결과 ===")
print("Best params:", study.best_params)
print("Best macro-F1:", study.best_value)

# === 최적 파라미터로 최종 학습 & Hugging Face 형식으로 저장 ===
best_lr         = study.best_params.get("lr", 2e-5)
best_epochs     = study.best_params.get("epochs", 3)
best_batch_size = study.best_params.get("batch_size", 16)
train_dataloader, val_dataloader = make_loaders(train_dataset, val_dataset, best_batch_size)

id2label = {0: "non_toxic", 1: "toxic"}
label2id = {"non_toxic": 0, "toxic": 1}
config = AutoConfig.from_pretrained(
    "klue/bert-base", num_labels=2, id2label=id2label, label2id=label2id
)
final_model = AutoModelForSequenceClassification.from_pretrained(
    "klue/bert-base", config=config
).to(device)

optimizer = optim.AdamW(final_model.parameters(), lr=best_lr)
criterion = nn.CrossEntropyLoss()

total_steps = len(train_dataloader) * best_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps
)

# ====== 설정: 출력/저장/로깅 ======
use_macro = True  # True면 macro P/R/F1, False면 binary(toxic 클래스 기준) 사용
EXPORT_DIR = "/content/drive/MyDrive/최종혐오진짜마지막"  # 원하면 경로 바꿔줘
os.makedirs(EXPORT_DIR, exist_ok=True)

# TensorBoard 설정
from torch.utils.tensorboard import SummaryWriter
TB_DIR = os.path.join(EXPORT_DIR, "tb_logs")
writer = SummaryWriter(TB_DIR)

# CSV 로그를 위한 버퍼
epoch_logs = []

# 표 헤더 출력
print(f"{'Epoch':>5}  {'Train Loss':>10}  {'Val Loss':>10}  {'Train Acc':>10}  {'Val Acc':>10}  "
      f"{'Train P':>8}  {'Train R':>8}  {'Train F1':>9}  {'Val P':>8}  {'Val R':>8}  {'Val F1':>8}")

best_f1 = 0.0
for ep in range(1, best_epochs + 1):
    tr = run_epoch_cls(
        final_model, train_dataloader, optimizer, criterion, device,
        train_mode=True, scheduler=scheduler
    )
    va = run_epoch_cls(
        final_model, val_dataloader, optimizer, criterion, device,
        train_mode=False, scheduler=None
    )

    # 지표 선택
    if use_macro:
        p_tr, r_tr, f1_tr = tr["p_macro"], tr["r_macro"], tr["f1_macro"]
        p_va, r_va, f1_va = va["p_macro"], va["r_macro"], va["f1_macro"]
    else:
        p_tr, r_tr, f1_tr = tr["p_bin"], tr["r_bin"], tr["f1_bin"]
        p_va, r_va, f1_va = va["p_bin"], va["r_bin"], va["f1_bin"]

    # 표 한 줄
    print(f"{ep:>5}  {tr['loss']:10.6f}  {va['loss']:10.6f}  {tr['acc']:10.6f}  {va['acc']:10.6f}  "
          f"{p_tr:8.6f}  {r_tr:8.6f}  {f1_tr:9.6f}  {p_va:8.6f}  {r_va:8.6f}  {f_va:8.6f}")

    # TensorBoard 로깅 (스칼라)
    writer.add_scalar("Loss/train", tr["loss"], ep)
    writer.add_scalar("Loss/valid", va["loss"], ep)
    writer.add_scalar("Acc/train", tr["acc"], ep)
    writer.add_scalar("Acc/valid", va["acc"], ep)
    writer.add_scalar(("F1_macro" if use_macro else "F1_bin") + "/train", f1_tr, ep)
    writer.add_scalar(("F1_macro" if use_macro else "F1_bin") + "/valid", f1_va, ep)
    writer.add_scalar(("Precision_macro" if use_macro else "Precision_bin") + "/train", p_tr, ep)
    writer.add_scalar(("Precision_macro" if use_macro else "Precision_bin") + "/valid", p_va, ep)
    writer.add_scalar(("Recall_macro" if use_macro else "Recall_bin") + "/train", r_tr, ep)
    writer.add_scalar(("Recall_macro" if use_macro else "Recall_bin") + "/valid", r_va, ep)
    # 현재 LR
    writer.add_scalar("LR", optimizer.param_groups[0]["lr"], ep)

    # CSV 로그 버퍼 적재
    epoch_logs.append({
        "epoch": ep,
        "train_loss": float(tr["loss"]), "val_loss": float(va["loss"]),
        "train_acc": float(tr["acc"]),   "val_acc": float(va["acc"]),
        "train_p": float(p_tr), "train_r": float(r_tr), "train_f1": float(f1_tr),
        "val_p": float(p_va),   "val_r": float(r_va),   "val_f1": float(f1_va)
    })

    # 베스트 저장(검증 macro-F1 기준 고정)
    if va["f1_macro"] > best_f1:
        best_f1 = va["f1_macro"]
        # HF 형식 저장 (가중치/설정/토크나이저)
        final_model.save_pretrained(EXPORT_DIR, safe_serialization=True)
        tokenizer.save_pretrained(EXPORT_DIR)
        # 메타 정보 저장
        with open(os.path.join(EXPORT_DIR, "meta.json"), "w", encoding="utf-8") as f:
            json.dump({
                "best_macro_f1": float(best_f1),
                "best_params": {
                    "lr": best_lr, "epochs": best_epochs, "batch_size": best_batch_size
                }
            }, f, ensure_ascii=False, indent=2)

# CSV 저장
import pandas as pd
csv_path = os.path.join(EXPORT_DIR, "epoch_metrics.csv")
pd.DataFrame(epoch_logs).to_csv(csv_path, index=False)

# 마무리
writer.flush()
writer.close()
print(f"\n🎯 최종 Best macro-F1: {best_f1:.6f}")
print(f"📦 저장 폴더: {EXPORT_DIR}")
print(f"📄 에폭 로그 CSV: {csv_path}")
print(f"📈 TensorBoard 로그: {TB_DIR}")






[I 2025-08-17 04:18:24,057] A new study created in memory with name: no-name-359c5d23-59d8-4a16-9f08-3a5458c91b9d


사용 디바이스: cuda


  0%|          | 0/20 [00:00<?, ?it/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[I 2025-08-17 04:25:35,082] Trial 0 finished with value: 0.9850186135777371 and parameters: {'lr': 1.9002736006237515e-05, 'epochs': 5, 'batch_size': 32}. Best is trial 0 with value: 0.9850186135777371.


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[I 2025-08-17 06:27:28,228] Trial 19 finished with value: 0.9847687310380662 and parameters: {'lr': 4.780410652932717e-05, 'epochs': 5, 'batch_size': 32}. Best is trial 16 with value: 0.9860169333017444.

=== HPO 결과 ===
Best params: {'lr': 1.6907410428011014e-05, 'epochs': 5, 'batch_size': 32}
Best macro-F1: 0.9860169333017444

=== 최적 하이퍼파라미터로 재학습 ===
lr=1.6907410428011014e-05, epochs=5, batch_size=32


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch  Train Loss    Val Loss   Train Acc     Val Acc   Train P   Train R   Train F1     Val P     Val R    Val F1


NameError: name 'f_va' is not defined

In [11]:
# =========================================
# 수동 재학습 (best 하이퍼파라미터 지정 버전)
# =========================================
import os, json, numpy as np, torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup
)

# ===== 0) 수동 지정한 최적 하이퍼파라미터 =====
best_lr         = 1.6907410428011014e-05
best_epochs     = 5
best_batch_size = 32

# ===== 1) 로더 생성 =====
def make_loaders(train_ds, val_ds, batch_size=16):
    return (
        DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=0),
        DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=0),
    )

train_dataloader, val_dataloader = make_loaders(train_dataset, val_dataset, best_batch_size)

# ===== 2) 모델 구성 =====
id2label = {0: "non_toxic", 1: "toxic"}
label2id = {"non_toxic": 0, "toxic": 1}
config = AutoConfig.from_pretrained(
    "klue/bert-base",
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)
final_model = AutoModelForSequenceClassification.from_pretrained(
    "klue/bert-base",
    config=config
).to(device)

optimizer = optim.AdamW(final_model.parameters(), lr=best_lr)
criterion = nn.CrossEntropyLoss()

total_steps = len(train_dataloader) * best_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps
)

# ===== 3) 에폭 실행 함수 (macro 지표 포함) =====
@torch.no_grad()
def _to_np(t): return t.detach().cpu().numpy()

def run_epoch_cls(model, dataloader, optimizer, criterion, device,
                  train_mode=True, scheduler=None, max_grad_norm=1.0):
    model.train() if train_mode else model.eval()
    total_loss = 0.0
    all_preds, all_labels = [], []

    for batch in dataloader:
        input_ids      = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels         = batch["label"].to(device)

        if train_mode:
            optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits  = outputs.logits
        loss    = criterion(logits, labels)

        if train_mode:
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            if scheduler is not None:
                scheduler.step()

        total_loss += loss.item() * labels.size(0)
        preds = torch.argmax(logits, dim=1)

        all_preds.append(_to_np(preds))
        all_labels.append(_to_np(labels))

    avg_loss  = total_loss / len(dataloader.dataset)
    y_pred    = np.concatenate(all_preds)
    y_true    = np.concatenate(all_labels)

    p_bin, r_bin, f1_bin, _ = precision_recall_fscore_support(y_true, y_pred, average='binary', zero_division=0)
    p_mac, r_mac, f1_mac, _ = precision_recall_fscore_support(y_true, y_pred, average='macro',  zero_division=0)
    acc = (y_pred == y_true).mean()
    cm  = confusion_matrix(y_true, y_pred, labels=[0,1]).tolist()

    return {
        "loss": avg_loss, "acc": acc,
        "p_bin": p_bin, "r_bin": r_bin, "f1_bin": f1_bin,
        "p_macro": p_mac, "r_macro": r_mac, "f1_macro": f1_mac,
        "cm": cm
    }

# ===== 4) 학습 루프 (표 출력 + 베스트 저장) =====
use_macro  = True   # True: macro P/R/F1 기준으로 표시
EXPORT_DIR = "/content/drive/MyDrive/최종혐오진짜마지막"  # 원하는 경로로 변경
os.makedirs(EXPORT_DIR, exist_ok=True)

print("\n=== 수동 재학습 시작 ===")
print(f"lr={best_lr}, epochs={best_epochs}, batch_size={best_batch_size}")
print(f"{'Ep':>3}  {'TrLoss':>7}  {'VaLoss':>7}  {'TrAcc':>7}  {'VaAcc':>7}  {'TrP':>6}  {'TrR':>6}  {'TrF1':>6}  {'VaP':>6}  {'VaR':>6}  {'VaF1':>6}")

best_f1 = 0.0
for ep in range(1, best_epochs + 1):
    tr = run_epoch_cls(final_model, train_dataloader, optimizer, criterion, device, True,  scheduler)
    va = run_epoch_cls(final_model, val_dataloader,   optimizer, criterion, device, False, None)

    if use_macro:
        p_tr, r_tr, f1_tr = tr["p_macro"], tr["r_macro"], tr["f1_macro"]
        p_va, r_va, f1_va = va["p_macro"], va["r_macro"], va["f1_macro"]
    else:
        p_tr, r_tr, f1_tr = tr["p_bin"], tr["r_bin"], tr["f1_bin"]
        p_va, r_va, f1_va = va["p_bin"], va["r_bin"], va["f1_bin"]

    print(f"{ep:>3}  {tr['loss']:7.4f}  {va['loss']:7.4f}  {tr['acc']:7.4f}  {va['acc']:7.4f}  "
          f"{p_tr:6.4f}  {r_tr:6.4f}  {f1_tr:6.4f}  {p_va:6.4f}  {r_va:6.4f}  {f1_va:6.4f}")

    # 베스트 저장 (검증 macro-F1 기준 고정)
    if va["f1_macro"] > best_f1:
        best_f1 = va["f1_macro"]
        final_model.save_pretrained(EXPORT_DIR, safe_serialization=True)  # model.safetensors
        tokenizer.save_pretrained(EXPORT_DIR)                             # tokenizer files
        with open(os.path.join(EXPORT_DIR, "meta.json"), "w", encoding="utf-8") as f:
            json.dump({
                "best_macro_f1": float(best_f1),
                "best_params": {
                    "lr": best_lr, "epochs": best_epochs, "batch_size": best_batch_size
                }
            }, f, ensure_ascii=False, indent=2)

print(f"\n🎯 최종 Best macro-F1: {best_f1:.6f}")
print(f"📦 저장 폴더: {EXPORT_DIR} (config.json, tokenizer, model.safetensors, meta.json)")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== 수동 재학습 시작 ===
lr=1.6907410428011014e-05, epochs=5, batch_size=32
 Ep   TrLoss   VaLoss    TrAcc    VaAcc     TrP     TrR    TrF1     VaP     VaR    VaF1
  1   0.1631   0.0673   0.9368   0.9818  0.9370  0.9368  0.9367  0.9818  0.9818  0.9818
  2   0.0397   0.0732   0.9888   0.9823  0.9888  0.9888  0.9888  0.9823  0.9823  0.9823
  3   0.0133   0.0901   0.9963   0.9830  0.9963  0.9963  0.9963  0.9830  0.9830  0.9830
  4   0.0044   0.0994   0.9990   0.9825  0.9990  0.9990  0.9990  0.9826  0.9825  0.9825
  5   0.0016   0.1061   0.9994   0.9823  0.9994  0.9994  0.9994  0.9823  0.9823  0.9823

🎯 최종 Best macro-F1: 0.983021
📦 저장 폴더: /content/drive/MyDrive/최종혐오진짜마지막 (config.json, tokenizer, model.safetensors, meta.json)


In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

EXPORT_DIR = "/content/drive/MyDrive/최종혐오진짜마지막"

tokenizer = AutoTokenizer.from_pretrained(EXPORT_DIR)
model = AutoModelForSequenceClassification.from_pretrained(EXPORT_DIR).to(device)

# 3. 맥락 기반 추론 함수 정의
def predict_conversation(model, tokenizer, conversation, target_index, device):

    model.eval()

    # 전체 대화를 하나의 텍스트로 합치기 (우리가 학습한 방식과 동일)
    full_conversation = " ".join(conversation)

    # 토큰화
    encoding = tokenizer(
        full_conversation,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt'
    )

    # 디바이스로 이동
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
      outputs = model(input_ids=input_ids, attention_mask=attention_mask)
      logits = outputs.logits if hasattr(outputs, "logits") else outputs
      probabilities = torch.softmax(logits, dim=1)
      prediction = torch.argmax(logits, dim=1)

    return {
        'conversation': conversation,
        'target_sentence': conversation[target_index],
        'prediction': prediction.item(),
        'label': 'toxic' if prediction.item() == 1 else 'non-toxic',
        'confidence': probabilities[0][prediction.item()].item(),
        'probabilities': {
            'non-toxic': probabilities[0][0].item(),
            'toxic': probabilities[0][1].item()
        }
    }

# 4. 실제 대화 맥락으로 테스트해보기 (간단히)
print("\n=== 맥락 기반 추론 테스트 ===")

test_conversations = [
    (["A: 나 다음주에 웨딩촬영해", "B: 오 진짜? 나도 갈게!", "A: 꼭 와서 나 웨딩드레스 입은 모습 봐줘 ㅎㅎ", "B: 너 웨딩드레스입은 모습 보면 완전 공주님 같을거야 ㅎㅎ"], 3),
    (["A: 오늘 점심 뭐 먹을까?", "B: 급식실 음식 어때?", "A: 괜찮을 것 같은데", "B: 좋아, 같이 가자"], 3)
]

for i, (conversation, target_idx) in enumerate(test_conversations, 1):
    result = predict_conversation(model, tokenizer, conversation, target_idx, device)
    print(f"테스트 {i}: '{result['target_sentence']}'")
    print(f"  → {result['label']} (신뢰도: {result['confidence']:.3f})")

# 5. 원본 데이터 3개만 확인
print(f"\n=== 원본 데이터 확인 ===")
for i in range(3):
    original_data = data[i]
    result = predict_conversation(model, tokenizer, original_data['utterance'], original_data['target_index'], device)
    actual = original_data['label']
    correct = "✅" if (actual == 'toxic' and result['prediction'] == 1) or (actual == 'non-toxic' and result['prediction'] == 0) else "❌"
    print(f"샘플 {i+1}: {actual} → {result['label']} {correct}")

# 6. 구글 드라이브에 모델 저장
print(f"\n=== 구글 드라이브에 모델 저장 ===")
drive_path = '/content/drive/MyDrive/hate_detection_model.pth'

# 전체 모델과 설정 저장
torch.save({
    'model_state_dict': model.state_dict(),
    'model_architecture': 'HateDetectionModel with KLUE-BERT',
    'training_info': {
        'epochs': 3,
        'best_val_accuracy': 0.9808,  # 여러분의 최고 성능
        'batch_size': 16,
        'learning_rate': 2e-5
    },
    'tokenizer_name': 'klue/bert-base',
    'max_length': 128,
    'data_info': {
        'total_samples': len(data),
        'train_samples': len(train_texts),
        'val_samples': len(val_texts)
    }
}, drive_path)

# 7. 저장 완료 메시지만
print(f"✅ 모델 저장 완료: {drive_path}")
print(f"✅ 맥락 기반 혐오표현 탐지 모델 완성!")


=== 맥락 기반 추론 테스트 ===
테스트 1: 'B: 너 웨딩드레스입은 모습 보면 완전 공주님 같을거야 ㅎㅎ'
  → non-toxic (신뢰도: 0.998)
테스트 2: 'B: 좋아, 같이 가자'
  → non-toxic (신뢰도: 1.000)

=== 원본 데이터 확인 ===
샘플 1: toxic → toxic ✅
샘플 2: non-toxic → toxic ❌
샘플 3: toxic → toxic ✅

=== 구글 드라이브에 모델 저장 ===
✅ 모델 저장 완료: /content/drive/MyDrive/hate_detection_model.pth
✅ 맥락 기반 혐오표현 탐지 모델 완성!


In [14]:
# 대화형 혐오표현 탐지 추론 함수

def interactive_hate_detection(model, tokenizer, device):
    """
    대화를 한 줄씩 입력받아서 마지막 발언의 혐오표현 여부를 판단하는 함수
    빈 줄 입력하면 종료하고 결과 출력
    """
    print("=== 대화형 혐오표현 탐지 ===")
    print("대화를 한 줄씩 입력하세요. (예: A: 안녕하세요)")
    print("빈 줄을 입력하면 마지막 발언을 분석합니다.")
    print("'quit'을 입력하면 종료합니다.\n")

    conversation = []

    while True:
        line = input(f"대화 {len(conversation)+1}: ").strip()

        if line.lower() == 'quit':
            print("종료합니다.")
            break

        if line == "":  # 빈 줄 입력시 분석 시작
            if len(conversation) == 0:
                print("대화가 없습니다. 다시 입력해주세요.\n")
                continue

            print(f"\n--- 입력된 대화 ---")
            for i, utterance in enumerate(conversation):
                print(f"{i+1}: {utterance}")

            # 마지막 발언 분석
            target_index = len(conversation) - 1
            result = predict_conversation(model, tokenizer, conversation, target_index, device)

            print(f"\n--- 분석 결과 ---")
            print(f"분석 대상: '{result['target_sentence']}'")
            print(f"결과: {result['label']}")
            print(f"신뢰도: {result['confidence']:.3f}")

            if result['prediction'] == 1:  # toxic
                print("⚠️  혐오/조롱 표현이 감지되었습니다.")
            else:
                print("✅ 정상적인 대화입니다.")

            # 새로운 대화 시작할지 묻기
            print(f"\n새로운 대화를 분석하시겠습니까? (y/n)")
            again = input().strip().lower()
            if again != 'y':
                print("종료합니다.")
                break
            else:
                conversation = []
                print("\n새로운 대화를 시작합니다.\n")
        else:
            conversation.append(line)

interactive_hate_detection(model, tokenizer, device)

=== 대화형 혐오표현 탐지 ===
대화를 한 줄씩 입력하세요. (예: A: 안녕하세요)
빈 줄을 입력하면 마지막 발언을 분석합니다.
'quit'을 입력하면 종료합니다.

대화 1: A: 오늘 어디 갈래?
대화 2: B: 같이 한강가는거 어때?
대화 3: A: 가는 길에 치킨 사가자!
대화 4: B: 또 먹어? 그러다 돼지된다 너 ㅋㅋㅋ
대화 5: 

--- 입력된 대화 ---
1: A: 오늘 어디 갈래?
2: B: 같이 한강가는거 어때?
3: A: 가는 길에 치킨 사가자!
4: B: 또 먹어? 그러다 돼지된다 너 ㅋㅋㅋ

--- 분석 결과 ---
분석 대상: 'B: 또 먹어? 그러다 돼지된다 너 ㅋㅋㅋ'
결과: toxic
신뢰도: 1.000
⚠️  혐오/조롱 표현이 감지되었습니다.

새로운 대화를 분석하시겠습니까? (y/n)
y

새로운 대화를 시작합니다.

대화 1: A: 어제 뭐했어?
대화 2: B: 나 그냥 친구들이랑 술먹었어
대화 3: A: 재밌었어?
대화 4: B: ㅇㅇ 오랜만에 보니까 스트레스 풀리고 좋았어
대화 5: 

--- 입력된 대화 ---
1: A: 어제 뭐했어?
2: B: 나 그냥 친구들이랑 술먹었어
3: A: 재밌었어?
4: B: ㅇㅇ 오랜만에 보니까 스트레스 풀리고 좋았어

--- 분석 결과 ---
분석 대상: 'B: ㅇㅇ 오랜만에 보니까 스트레스 풀리고 좋았어'
결과: non-toxic
신뢰도: 1.000
✅ 정상적인 대화입니다.

새로운 대화를 분석하시겠습니까? (y/n)
y

새로운 대화를 시작합니다.

대화 1: A: 오늘 어디 갈래?
대화 2: B: 같이 한강가는거 어때?
대화 3: A: 가는 길에 치킨 사가자!
대화 4: B: 밥먹은지 얼마 안됐으니 조금 줄이는건 어떨까
대화 5: 

--- 입력된 대화 ---
1: A: 오늘 어디 갈래?
2: B: 같이 한강가는거 어때?
3: A: 가는 길에 치킨 사가자!
4: B: 밥먹은지 얼마 안됐으니 조금 줄이는건 어떨까

--- 분석 결과 ---
분석 대상: 'B: 밥먹은지 얼마 

KeyboardInterrupt: Interrupted by user

In [35]:
# =========================================
# 성능 시각화 3종 세트 (학습곡선, 혼동행렬, PR 커브)
# 전제: device, tokenizer, (val_dataloader 또는 val_texts/val_labels/HateDataset) 사용 가능
# =========================================
import os, json, numpy as np, pandas as pd, torch
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_curve, average_precision_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ===== 경로 설정 (네가 저장한 위치로 수정 가능) =====
EXPORT_DIR = "/content/drive/MyDrive/최종혐오진짜마지막"
os.makedirs(EXPORT_DIR, exist_ok=True)

# ===== 0) 모델/토크나이저 불러오기 (HF 포맷) =====
tokenizer = AutoTokenizer.from_pretrained(EXPORT_DIR)
model = AutoModelForSequenceClassification.from_pretrained(EXPORT_DIR).to(device)
model.eval()

# ===== 1) 학습 곡선 (epoch_metrics.csv → Loss/F1) =====
csv_path = os.path.join(EXPORT_DIR, "epoch_metrics.csv")
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)

    # Loss 곡선
    plt.figure()
    plt.plot(df["epoch"], df["train_loss"], label="Train Loss")
    plt.plot(df["epoch"], df["val_loss"],   label="Val Loss")
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title("Loss per Epoch"); plt.legend()
    loss_png = os.path.join(EXPORT_DIR, "plot_loss_per_epoch.png")
    plt.savefig(loss_png, bbox_inches="tight", dpi=200)
    plt.close()

    # F1(macro) 곡선
    plt.figure()
    plt.plot(df["epoch"], df["train_f1"], label="Train F1(macro)")
    plt.plot(df["epoch"], df["val_f1"],   label="Val F1(macro)")
    plt.xlabel("Epoch"); plt.ylabel("F1(macro)"); plt.title("F1(macro) per Epoch"); plt.legend()
    f1_png = os.path.join(EXPORT_DIR, "plot_f1_per_epoch.png")
    plt.savefig(f1_png, bbox_inches="tight", dpi=200)
    plt.close()

    print("✅ 학습곡선 저장:", loss_png, f1_png)
else:
    print("⚠️ epoch_metrics.csv 를 찾을 수 없습니다 → 학습곡선 스킵")

# ===== 2) 혼동행렬 & 3) PR 커브 =====
# val_dataloader 가 있으면 그대로 사용. 없으면 val_texts/val_labels로 재구성 시도.
def collect_val_batches():
    if "val_dataloader" in globals() and val_dataloader is not None:
        return val_dataloader

    # 없으면 재구성 (val_texts/val_labels/HateDataset 가 있어야 함)
    if "val_texts" in globals() and "val_labels" in globals() and "HateDataset" in globals():
        tmp_ds = HateDataset(val_texts, val_labels, tokenizer)
        from torch.utils.data import DataLoader
        return DataLoader(tmp_ds, batch_size=32, shuffle=False, num_workers=0)

    raise RuntimeError("val_dataloader가 없고, (val_texts/val_labels/HateDataset)도 없어 재구성 불가합니다.")

val_loader = collect_val_batches()

y_true, y_pred, y_score = [], [], []
with torch.no_grad():
    for batch in val_loader:
        input_ids      = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels         = batch["label"].cpu().numpy()
        outputs        = model(input_ids=input_ids, attention_mask=attention_mask)
        logits         = outputs.logits
        probs          = torch.softmax(logits, dim=1)[:, 1]         # toxic=1 확률
        preds          = torch.argmax(logits, dim=1).cpu().numpy()

        y_true.append(labels)
        y_pred.append(preds)
        y_score.append(probs.cpu().numpy())

y_true  = np.concatenate(y_true)
y_pred  = np.concatenate(y_pred)
y_score = np.concatenate(y_score)

# ===== (2) 혼동행렬 — pro 스타일(흰 배경, % + 개수 동시 표기) =====
cm = confusion_matrix(y_true, y_pred, labels=[0, 1])  # [[TN,FP],[FN,TP]]
cm_norm = cm / cm.sum(axis=1, keepdims=True)

fig, ax = plt.subplots(figsize=(6.6, 6.2))
im = ax.imshow(cm_norm, cmap="viridis", interpolation="nearest", aspect="equal")

# 축/제목(바깥 글자) 검정색
ax.set_title("Confusion Matrix — Validation", fontsize=16, color="black")
ax.set_xlabel("Predicted label", fontsize=12, color="black")
ax.set_ylabel("True label", fontsize=12, color="black")
ax.set_xticks([0, 1]); ax.set_yticks([0, 1])
ax.set_xticklabels(["non-toxic(0)", "toxic(1)"], color="black")
ax.set_yticklabels(["non-toxic(0)", "toxic(1)"], color="black")

# 칸마다 퍼센트 + 개수 함께 표기 (밝기 기준으로 글자색 자동 전환)
for i in range(2):
    for j in range(2):
        pct = f"{cm_norm[i, j]*100:.1f}%"
        cnt = f"{cm[i, j]:,}"
        # viridis 기준 임계(대략 0.5)로 밝으면 흰 글자
        text_color = "black" if cm_norm[i, j] > 0.5 else "white"
        ax.text(j, i, f"{pct}\n{cnt}", ha="center", va="center",
                fontsize=14, fontweight="bold", color=text_color)

# 컬러바(라벨 검정)
cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
cbar.ax.set_ylabel("Proportion", rotation=270, labelpad=12, color="black")
# 컬러바 눈금도 검정
for t in cbar.ax.get_yticklabels():
    t.set_color("black")

# 타일 경계 그리드
import numpy as np
ax.set_xticks(np.arange(-.5, 2, 1), minor=True)
ax.set_yticks(np.arange(-.5, 2, 1), minor=True)
ax.grid(which="minor", color="white", alpha=0.3, linewidth=1)
ax.tick_params(which="minor", bottom=False, left=False)

plt.tight_layout()
cm_png = os.path.join(EXPORT_DIR, "plot_confusion_matrix_pretty.png")
plt.savefig(cm_png, bbox_inches="tight", facecolor="white")  # 흰 배경 확정
plt.close()

print("✅ 혼동행렬 저장:", cm_png, " | CM =", cm.tolist())

# (3) PR 커브 (toxic=positive)
precision, recall, thresholds = precision_recall_curve(y_true, y_score, pos_label=1)
ap = average_precision_score(y_true, y_score)
plt.figure()
plt.plot(recall, precision, label=f"PR curve (AP={ap:.3f})")
plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title("Precision–Recall Curve (toxic=1)")
plt.legend()
pr_png = os.path.join(EXPORT_DIR, "plot_precision_recall_curve.png")
plt.savefig(pr_png, bbox_inches="tight", dpi=200)
plt.close()
print("✅ PR 커브 저장:", pr_png)

# (선택) PR 포인트 CSV도 저장해두면 나중에 커스텀 그래프 만들기 편함
pr_csv = os.path.join(EXPORT_DIR, "pr_curve_points.csv")
pd.DataFrame({"precision": precision, "recall": recall}).to_csv(pr_csv, index=False)
print("📄 PR 포인트 CSV:", pr_csv)


✅ 학습곡선 저장: /content/drive/MyDrive/최종혐오진짜마지막/plot_loss_per_epoch.png /content/drive/MyDrive/최종혐오진짜마지막/plot_f1_per_epoch.png
✅ 혼동행렬 저장: /content/drive/MyDrive/최종혐오진짜마지막/plot_confusion_matrix_pretty.png  | CM = [[1969, 34], [34, 1968]]
✅ PR 커브 저장: /content/drive/MyDrive/최종혐오진짜마지막/plot_precision_recall_curve.png
📄 PR 포인트 CSV: /content/drive/MyDrive/최종혐오진짜마지막/pr_curve_points.csv


In [27]:
import os, pandas as pd, matplotlib.pyplot as plt
OUT_DIR = "/content/drive/MyDrive/최종혐오진짜마지막"
os.makedirs(OUT_DIR, exist_ok=True)

# 🔧 여기에 표 숫자를 채워 넣어 (예시는 자리표시자)
data = {
    "epoch":     [1,2,3,4,5],
    "train_loss":[0.1631,0.0397,0.0133,0.0044,0.0016],
    "val_loss":  [0.0673,0.0732,0.0901,0.0994,0.1061],
    "train_acc": [0.9368,0.9688,0.9963,0.9990,0.9994],
    "val_acc":   [0.9818,0.9823,0.9823,0.9825,0.9823],
    "train_p":   [0.9370,0.9880,0.9963,0.9990,0.9994],
    "train_r":   [0.9368,0.9880,0.9963,0.9990,0.9994],
    "train_f1":  [0.9367,0.9880,0.9963,0.9990,0.9994],
    "val_p":     [0.9818,0.9823,0.9830,0.9825,0.9823],
    "val_r":     [0.9818,0.9823,0.9830,0.9825,0.9823],
    "val_f1":    [0.9818,0.9823,0.9830,0.9825,0.9823],
}
df = pd.DataFrame(data)
csv_path = os.path.join(OUT_DIR, "epoch_metrics.csv")
df.to_csv(csv_path, index=False)
print("✅ 수동 CSV 저장:", csv_path)

plt.style.use("dark_background"); plt.rcParams.update({"figure.dpi":160,"savefig.dpi":300})
def plot(x,y1,y2,title,ylabel,fname):
    fig,ax=plt.subplots(figsize=(7.2,4.2))
    ax.plot(x,y1,label="Train",linewidth=2); ax.plot(x,y2,label="Valid",linewidth=2)
    ax.set_title(title); ax.set_xlabel("Epoch"); ax.set_ylabel(ylabel); ax.grid(alpha=.25); ax.legend()
    out=os.path.join(OUT_DIR,fname); plt.tight_layout(); plt.savefig(out, bbox_inches="tight", transparent=True); plt.close()
    print("📈 저장:", out)

plot(df["epoch"], df["train_loss"], df["val_loss"], "Loss per Epoch", "Loss", "curve_loss.png")
plot(df["epoch"], df["train_acc"],  df["val_acc"],  "Accuracy per Epoch", "Accuracy", "curve_acc.png")
plot(df["epoch"], df["train_f1"],   df["val_f1"],   "F1(macro) per Epoch", "F1(macro)", "curve_f1.png")


✅ 수동 CSV 저장: /content/drive/MyDrive/최종혐오진짜마지막/epoch_metrics.csv
📈 저장: /content/drive/MyDrive/최종혐오진짜마지막/curve_loss.png
📈 저장: /content/drive/MyDrive/최종혐오진짜마지막/curve_acc.png
📈 저장: /content/drive/MyDrive/최종혐오진짜마지막/curve_f1.png


In [29]:
# ============================
# Pro-style single charts (white)
# ============================
import os, numpy as np, matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, precision_recall_curve, average_precision_score

# 저장 폴더
OUT_DIR = "/content/drive/MyDrive/최종혐오진짜마지막"
os.makedirs(OUT_DIR, exist_ok=True)

# ===== 0) 공통 스타일(흰 배경, 그리드, 폰트) =====
plt.style.use("default")
plt.rcParams.update({
    "figure.dpi": 160, "savefig.dpi": 300,
    "font.size": 11,
    "axes.titlesize": 15,
    "axes.labelsize": 12,
    "axes.spines.top": False,
    "axes.spines.right": False,
    "axes.grid": True,
    "grid.alpha": 0.25,
    "legend.frameon": False,
})

# ===== 1) 표 숫자 (네가 캡처에서 본 값) =====
epochs    = np.array([1,2,3,4,5])
train_loss= np.array([0.1631, 0.0397, 0.0133, 0.0044, 0.0016])
val_loss  = np.array([0.0673, 0.0732, 0.0901, 0.0994, 0.1061])
train_acc = np.array([0.9368, 0.9688, 0.9963, 0.9990, 0.9994])
val_acc   = np.array([0.9818, 0.9823, 0.9823, 0.9825, 0.9823])
train_f1  = np.array([0.9367, 0.9880, 0.9963, 0.9990, 0.9994])
val_f1    = np.array([0.9818, 0.9823, 0.9830, 0.9825, 0.9823])

# ===== 2) Loss 곡선 (단일 차트) =====
def plot_loss_white(ep, tr, va, out_path):
    fig, ax = plt.subplots(figsize=(7.2, 4.2))
    ax.plot(ep, tr, marker="o", linewidth=2, label="Train Loss")
    ax.plot(ep, va, marker="o", linewidth=2, label="Validation Loss")
    ax.set_title("Loss per Epoch")
    ax.set_xlabel("Epoch"); ax.set_ylabel("Loss")

    # 핵심 포인트 주석: 최소 Val Loss
    min_idx = int(np.argmin(va))
    ax.scatter(ep[min_idx], va[min_idx], s=60, zorder=5)
    ax.annotate(f"min Val={va[min_idx]:.4f} @Epoch {ep[min_idx]}",
                xy=(ep[min_idx], va[min_idx]),
                xytext=(ep[min_idx]+0.2, va[min_idx]+(va.max()-va.min())*0.25),
                arrowprops=dict(arrowstyle="->", lw=1.2))

    ax.legend(loc="upper right", bbox_to_anchor=(1.0, 1.05))
    fig.tight_layout()
    fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)

# ===== 3) Accuracy 곡선 (단일 차트) =====
def plot_acc_white(ep, tr, va, out_path):
    fig, ax = plt.subplots(figsize=(7.2, 4.2))
    ax.plot(ep, tr, marker="s", linewidth=2, label="Train Accuracy")
    ax.plot(ep, va, marker="s", linewidth=2, label="Validation Accuracy")
    ax.set_title("Accuracy per Epoch")
    ax.set_xlabel("Epoch"); ax.set_ylabel("Accuracy")
    ax.set_ylim(0.92, 1.005)

    # 간격 레이블 보기 좋게
    ax.yaxis.set_major_formatter(lambda x, pos: f"{x:.3f}")
    ax.legend(loc="lower right", bbox_to_anchor=(1.0, -0.02), ncol=2)
    fig.tight_layout()
    fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)

# ===== 4) F1(macro) 곡선 (단일 차트) =====
def plot_f1_white(ep, tr, va, out_path):
    fig, ax = plt.subplots(figsize=(7.2, 4.2))
    ax.plot(ep, tr, marker="^", linewidth=2, label="Train F1 (macro)")
    ax.plot(ep, va, marker="^", linewidth=2, label="Validation F1 (macro)")
    ax.set_title("F1(macro) per Epoch")
    ax.set_xlabel("Epoch"); ax.set_ylabel("F1 (macro)")
    ax.set_ylim(0.92, 1.005)

    # 핵심 포인트 주석: 최대 Val F1
    max_idx = int(np.argmax(va))
    ax.scatter(ep[max_idx], va[max_idx], s=60, zorder=5)
    ax.annotate(f"max Val={va[max_idx]:.4f} @Epoch {ep[max_idx]}",
                xy=(ep[max_idx], va[max_idx]),
                xytext=(ep[max_idx]-1.2, va[max_idx]-(va.max()-va.min())*0.25),
                arrowprops=dict(arrowstyle="->", lw=1.2))

    ax.legend(loc="lower right", bbox_to_anchor=(1.0, -0.02), ncol=2)
    fig.tight_layout()
    fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)

# ===== 5) Confusion Matrix (흰 배경, 정규화% + 개수 동시 표기) =====
def plot_confusion_white(y_true, y_pred, labels=("non-toxic(0)","toxic(1)"), out_path=None):
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_true, y_pred, labels=[0,1])
    cm_norm = cm / cm.sum(axis=1, keepdims=True)

    fig, ax = plt.subplots(figsize=(6.2, 6.2))
    im = ax.imshow(cm_norm, cmap="Blues", interpolation="nearest", aspect="equal")
    ax.set_title("Confusion Matrix — Validation")
    ax.set_xlabel("Predicted label"); ax.set_ylabel("True label")
    ax.set_xticks([0,1]); ax.set_yticks([0,1])
    ax.set_xticklabels(labels); ax.set_yticklabels(labels)

    # 칸마다 퍼센트+개수
    for i in range(2):
        for j in range(2):
            ax.text(j, i, f"{cm_norm[i,j]*100:.1f}%\n{cm[i,j]:,}", ha="center", va="center",
                    fontsize=11, fontweight="bold", color=("black" if cm_norm[i,j] < 0.6 else "white"))

    cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.ax.set_ylabel("Proportion", rotation=270, labelpad=12)
    ax.set_xticks(np.arange(-.5, 2, 1), minor=True)
    ax.set_yticks(np.arange(-.5, 2, 1), minor=True)
    ax.grid(which="minor", color="gray", alpha=0.2, linewidth=1)
    ax.tick_params(which="minor", bottom=False, left=False)

    fig.tight_layout()
    if out_path:
        fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)
    return cm

# ===== 6) PR Curve (흰 배경, AP & 베스트 F1 포인트 표시) =====
def plot_pr_white(y_true, y_score, out_path=None, pos_label=1):
    from sklearn.metrics import precision_recall_curve, average_precision_score
    precision, recall, _ = precision_recall_curve(y_true, y_score, pos_label=pos_label)
    ap = average_precision_score(y_true, y_score)

    # F1 최대
    f1s = (2*precision*recall)/(precision+recall+1e-9)
    best = int(np.nanargmax(f1s))

    fig, ax = plt.subplots(figsize=(7.2, 4.4))
    ax.plot(recall, precision, linewidth=2, label=f"PR curve • AP={ap:.3f}")
    ax.scatter([recall[best]], [precision[best]], s=60, zorder=5)
    ax.annotate(f"Best F1={f1s[best]:.3f}\nP={precision[best]:.3f}, R={recall[best]:.3f}",
                xy=(recall[best], precision[best]),
                xytext=(min(0.85, recall[best]+0.12), min(0.98, precision[best]+0.08)),
                arrowprops=dict(arrowstyle='->', lw=1.2))
    ax.set_xlim(0,1); ax.set_ylim(0,1.02)
    ax.set_title("Precision–Recall (toxic=positive)")
    ax.set_xlabel("Recall"); ax.set_ylabel("Precision")
    ax.legend(loc="lower left")
    fig.tight_layout()
    if out_path:
        fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)

# ===== 7) 실제 저장 =====
plot_loss_white(epochs, train_loss, val_loss, os.path.join(OUT_DIR, "chart_loss_white.png"))
plot_acc_white(epochs, train_acc, val_acc,   os.path.join(OUT_DIR, "chart_acc_white.png"))
plot_f1_white(epochs, train_f1, val_f1,     os.path.join(OUT_DIR, "chart_f1_white.png"))

print("✅ Saved:",
      os.path.join(OUT_DIR, "chart_loss_white.png"),
      os.path.join(OUT_DIR, "chart_acc_white.png"),
      os.path.join(OUT_DIR, "chart_f1_white.png"))

# 아래 둘은 원하면 넣어 (검증 결과가 y_true, y_pred, y_score 로 준비되어 있어야 함)
# cm = plot_confusion_white(y_true, y_pred, out_path=os.path.join(OUT_DIR, "chart_confmat_white.png"))
# plot_pr_white(y_true, y_score, out_path=os.path.join(OUT_DIR, "chart_pr_white.png"))


✅ Saved: /content/drive/MyDrive/최종혐오진짜마지막/chart_loss_white.png /content/drive/MyDrive/최종혐오진짜마지막/chart_acc_white.png /content/drive/MyDrive/최종혐오진짜마지막/chart_f1_white.png


In [33]:
import os, numpy as np, matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from google.colab import files

# 코랩 로컬 저장 위치
OUT = "/content/pretty_confmat.png"

def plot_confusion_matrix_pretty(y_true, y_pred, labels=("non-toxic(0)", "toxic(1)"), save_path=None):
    plt.style.use("default")
    plt.rcParams.update({
        "figure.dpi": 160,
        "savefig.dpi": 300,
        "font.size": 12,
        "axes.titlesize": 16,
        "axes.labelsize": 12,
        "legend.frameon": False,
        "axes.spines.top": False,
        "axes.spines.right": False,
    })

    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    cm_norm = cm / cm.sum(axis=1, keepdims=True)

    fig, ax = plt.subplots(figsize=(6.6, 6.2))
    im = ax.imshow(cm_norm, cmap="viridis", interpolation="nearest", aspect="equal")

    ax.set_title("Confusion Matrix — Validation")
    ax.set_xlabel("Predicted label"); ax.set_ylabel("True label")
    ax.set_xticks([0, 1]); ax.set_yticks([0, 1])
    ax.set_xticklabels(labels); ax.set_yticklabels(labels)

    # 칸마다 퍼센트 + 개수 표시 (밝은 칸엔 흰 글자)
    for i in range(2):
        for j in range(2):
            pct = f"{cm_norm[i, j]*100:.1f}%"
            cnt = f"{cm[i, j]:,}"
            color = "white" if cm_norm[i, j] > 0.5 else "black"
            ax.text(j, i, f"{pct}\n{cnt}", ha="center", va="center",
                    fontsize=14, fontweight="bold", color=color)

    # 컬러바
    cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.ax.set_ylabel("Proportion", rotation=270, labelpad=12)

    # 타일 경계 그리드
    ax.set_xticks(np.arange(-.5, 2, 1), minor=True)
    ax.set_yticks(np.arange(-.5, 2, 1), minor=True)
    ax.grid(which="minor", color="white", alpha=0.25, linewidth=1)
    ax.tick_params(which="minor", bottom=False, left=False)

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, bbox_inches="tight", facecolor="white")  # 흰 배경으로 저장
    plt.close(fig)

# 예시: 실제 검증 라벨/예측으로 호출
# val_labels = ...
# val_preds  = ...
# plot_confusion_matrix_pretty(val_labels, val_preds, save_path=OUT)
# files.download(OUT)
